In [1]:
import json
import pprint
import time

from IPython.display import HTML, display
import requests
import tabulate


def pretty_print_result(result):
    return display(HTML(tabulate.tabulate(result, headers='keys', tablefmt='html')))

In [2]:
# Get all regions' names.
query = '''{
    Region {
        name @output(out_name: "region_name")
    }
}'''

args = {}

In [3]:
query = '''{
    NobleHouse {
        name @output(out_name: "house") @tag(tag_name: "h")
        
        out_Has_Seat {
            name @filter(op_name: "has_substring", value: ["%h"]) @output(out_name: "region")
        }
    }
}'''

args = {}

In [4]:
# Get all the parent regions of the "Great Sept of Baelor" region.
query = '''{
    Region @filter(op_name: "name_or_alias", value: ["$region"]) {
        out_Has_Parent_Region {
            out_Has_Parent_Region @recurse(depth: 10) {
                name @output(out_name: "parent_region")
            }
        }
    }
}'''

args = {
    'region': 'Great Sept of Baelor'
}

In [5]:
# Get all the people that live in The Riverlands.
query = '''{
    Region @filter(op_name: "name_or_alias", value: ["$region"]) {        
        in_Has_Parent_Region @recurse(depth: 10) {
            name @output(out_name: "region_name")
            
            in_Lives_In {
                name @output(out_name: "character_name")
                
                out_Owes_Allegiance_To @fold {
                    ... on NobleHouse {
                        name @output(out_name: "loyal_to_noble_houses")
                    }
                }
            }
        }
    }
}'''

args = {
    'region': 'The Riverlands',
}

In [6]:
# Get everyone that Sansa Stark owes allegiance to.
query = '''{
    Character @filter(op_name: "name_or_alias", value: ["$character"]) {
        out_Owes_Allegiance_To @fold {
            __typename @output(out_name: "allegiance_type")
            name @output(out_name: "allegiances")
        }
    }
}'''

args = {
    'character': 'Sansa Stark',
}

In [7]:
# Get all the characters with allegiances to more than one noble house.
query = '''{
    Character {
        name @output(out_name: "character")
        out_Owes_Allegiance_To @fold {
            ... on NobleHouse {
                _x_count @filter(op_name: ">=", value: ["$min_allegiances"])
                name @output(out_name: "allegiances")
            }
        }
    }
}'''

args = {
    'min_allegiances': 2
}

In [ ]:
payload = {
    'query': query,
    'args': args,
}

start = time.time()
response = requests.post('http://localhost:5000/graphql', data=json.dumps(payload))
end = time.time()

print(response.text)

response_data = json.loads(response.text)
print('Received response in: {}s'.format(end - start))

print('\nInput metadata:')
pprint.pprint(response_data['input_metadata'])

print('\nOutput metadata:')
pprint.pprint(response_data['output_metadata'])

print('\nOutput data:')
pretty_print_result(response_data['output_data'])